# Prompt Optimisation

In [3]:
import os
import logging
import mlflow
import openai

from dotenv import load_dotenv
from mlflow.genai.optimize import GepaPromptOptimizer
from mlflow.genai.scorers import Correctness

# load_dotenv("../.env")
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_registry_uri("http://127.0.0.1:5000")

## Training Data

In [4]:
raw_data = [
    ("The emergence of HIV as a chronic condition means that people living with HIV are required to take more responsibility for the self-management of their condition , including making physical , emotional and social adjustments .", "BACKGROUND"),
    ("This paper describes the design and evaluation of Positive Outlook , an online program aiming to enhance the self-management skills of gay men living with HIV .", "BACKGROUND"),
    ("This study is designed as a randomised controlled trial in which men living with HIV in Australia will be assigned to either an intervention group or usual care control group .", "METHODS"),
    ("The intervention group will participate in the online group program ` Positive Outlook ' .", "METHODS"),
    ("The program is based on self-efficacy theory and uses a self-management approach to enhance skills , confidence and abilities to manage the psychosocial issues associated with HIV in daily life .", "METHODS"),
    ("Participants will access the program for a minimum of 90 minutes per week over seven weeks .", "METHODS"),
    ("Primary outcomes are domain specific self-efficacy , HIV related quality of life , and outcomes of health education .", "METHODS"),
    ("Secondary outcomes include : depression , anxiety and stress ; general health and quality of life ; adjustment to HIV ; and social support .", "METHODS"),
    ("Data collection will take place at baseline , completion of the intervention ( or eight weeks post randomisation ) and at 12 week follow-up .", "METHODS"),
    ("Results of the Positive Outlook study will provide information regarding the effectiveness of online group programs improving health related outcomes for men living with HIV .", "CONCLUSIONS"),
    ("The aim of this study was to evaluate the efficacy , safety and complications of orbital steroid injection versus oral steroid therapy in the management of thyroid-related ophthalmopathy .", "OBJECTIVE"),
    ("A total of 29 patients suffering from thyroid ophthalmopathy were included in this study .", "METHODS"),
    ("Patients were randomized into two groups : group I included 15 patients treated with oral prednisolone and group II included 14 patients treated with peribulbar triamcinolone orbital injection .", "METHODS"),
    ("Both groups showed improvement in symptoms and in clinical evidence of inflammation with improvement of eye movement and proptosis in most cases .", "RESULTS"),
    ("Mean exophthalmometry value before treatment was 22.6 1.98 mm that decreased to 18.6 0.996 mm in group I , compared with 23 1.86 mm that decreased to 19.08 1.16 mm in group II .", "RESULTS"),
    ("There was no change in the best-corrected visual acuity in both groups .", "RESULTS"),
    ("There was an increase in body weight , blood sugar , blood pressure and gastritis in group I in 66.7 % , 33.3 % , 50 % and 75 % , respectively , compared with 0 % , 0 % , 8.3 % and 8.3 % in group II .", "RESULTS"),
    ("Orbital steroid injection for thyroid-related ophthalmopathy is effective and safe .", "CONCLUSIONS"),
    ("It eliminates the adverse reactions associated with oral corticosteroid use .", "CONCLUSIONS"),
    ("The aim of this prospective randomized study was to examine whether active counseling and more liberal oral fluid intake decrease postoperative pain , nausea and vomiting in pediatric ambulatory tonsillectomy .", "OBJECTIVE"),
]

dataset = [
    {
        "inputs": {"sentence": sentence},
        "expectations": {"expected_response": label},
    }
    for sentence, label in raw_data
]

logger.info(f"# of Sample: {len(dataset)}")

INFO:__main__:# of Sample: 20


## Registering the base prompt

In [5]:
prompt = mlflow.genai.register_prompt(
    name="medical_section_classifier",
    template="Classify this medical research paper sentence into one of these sections: CONCLUSIONS, RESULTS, METHODS, OBJECTIVE, BACKGROUND.\n\nSentence: {{sentence}}",
)

2025/11/20 15:29:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: medical_section_classifier, version 1


## Agent Execution / Inference

In [6]:
def predict_fn(sentence: str) -> str:
    _prompt = mlflow.genai.load_prompt("prompts:/medical_section_classifier/1")
    completion = openai.OpenAI().chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role": "user", "content": prompt.format(sentence=sentence)}],
    )
    return completion.choices[0].message.content

## Setting the Experiment

In [7]:
experiment_name = "dummy_agent_prompt_optimisation"
try:
    experiment = mlflow.get_experiment_by_name(experiment_name)
    if experiment is None:
        # Create new experiment if it doesn't exist
        experiment_id = mlflow.create_experiment(experiment_name)
    else:
        experiment_id = experiment.experiment_id

    # Set the experiment
    mlflow.set_experiment(experiment_name)
except Exception as e:
    print(f"Error setting up experiment: {e}")
    # Fallback: create a default experiment
    experiment_id = mlflow.create_experiment("default")
    mlflow.set_experiment("default")

# Ensure MLflow is properly initialized before optimization
print(f"Using experiment: {mlflow.get_experiment(experiment_id).name}")
print(f"MLflow tracking URI: {mlflow.get_tracking_uri()}")


Using experiment: dummy_agent_prompt_optimisation
MLflow tracking URI: http://127.0.0.1:5000


## Optimisation Loop

In [6]:
# Optimize the prompt
result = mlflow.genai.optimize_prompts(
    predict_fn=predict_fn,
    train_data=dataset,
    prompt_uris=[prompt.uri],
    optimizer=GepaPromptOptimizer(
        reflection_model="openai:/gpt-5",
        max_metric_calls=300,
        display_progress_bar=True,
    ),
    scorers=[Correctness(model="openai:/gpt-5-mini")],
)

# Use the optimized prompt
optimized_prompt = result.optimized_prompts[0]
print(f"Optimized template: {optimized_prompt.template}")

2025/11/11 14:00:38 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.
2025/11/11 14:00:38 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset. To disable this check, set the MLFLOW_GENAI_EVAL_SKIP_TRACE_VALIDATION environment variable to True.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/sangramreddy/Documents/dev/pocs/hobu-poi-service/.venv/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:148: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for 'prompt_optimization_train_data'. Exception: 
  return _dataset_source_registry.resolve(
/Users/sangramreddy/Documents/dev/pocs/hobu-poi-service/.venv/lib/python3.12/site-packages/mlflow/data/dataset_source_registry.py:148: U

Iteration 0: Base program full valset score: 0.9 over 20 / 20 examples
Iteration 1: Selected program 0 score: 0.9


14:01:08 - LiteLLM:INFO: utils.py:3347 - 
LiteLLM completion() model= gpt-5-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5-mini; provider = openai
14:01:09 - LiteLLM:INFO: utils.py:3347 - 
LiteLLM completion() model= gpt-5-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5-mini; provider = openai
14:01:09 - LiteLLM:INFO: utils.py:3347 - 
LiteLLM completion() model= gpt-5-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5-mini; provider = openai
14:01:11 - LiteLLM:INFO: utils.py:1273 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
14:01:12 - LiteLLM:INFO: utils.py:1273 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
14:01:16 - LiteLLM:INFO: utils.py:1273 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


Iteration 1: All subsample scores perfect. Skipping.
Iteration 1: Reflective mutation did not propose a new candidate
Iteration 2: Selected program 0 score: 0.9


14:01:19 - LiteLLM:INFO: utils.py:3347 - 
LiteLLM completion() model= gpt-5-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5-mini; provider = openai
14:01:20 - LiteLLM:INFO: utils.py:3347 - 
LiteLLM completion() model= gpt-5-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5-mini; provider = openai
14:01:20 - LiteLLM:INFO: utils.py:3347 - 
LiteLLM completion() model= gpt-5-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5-mini; provider = openai
14:01:23 - LiteLLM:INFO: utils.py:1273 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
14:01:24 - LiteLLM:INFO: utils.py:1273 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
14:01:27 - LiteLLM:INFO: utils.py:1273 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


...

Iteration 46: All subsample scores perfect. Skipping.
Iteration 46: Reflective mutation did not propose a new candidate
Iteration 47: Selected program 5 score: 1.0


14:23:27 - LiteLLM:INFO: utils.py:3347 - 
LiteLLM completion() model= gpt-5-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5-mini; provider = openai
14:23:27 - LiteLLM:INFO: utils.py:3347 - 
LiteLLM completion() model= gpt-5-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5-mini; provider = openai
14:23:29 - LiteLLM:INFO: utils.py:3347 - 
LiteLLM completion() model= gpt-5-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5-mini; provider = openai
14:23:32 - LiteLLM:INFO: utils.py:1273 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
14:23:34 - LiteLLM:INFO: utils.py:1273 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
14:23:37 - LiteLLM:INFO: utils.py:1273 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


Iteration 47: All subsample scores perfect. Skipping.
Iteration 47: Reflective mutation did not propose a new candidate
Iteration 48: Selected program 5 score: 1.0


14:23:39 - LiteLLM:INFO: utils.py:3347 - 
LiteLLM completion() model= gpt-5-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5-mini; provider = openai
14:23:39 - LiteLLM:INFO: utils.py:3347 - 
LiteLLM completion() model= gpt-5-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5-mini; provider = openai
14:23:39 - LiteLLM:INFO: utils.py:3347 - 
LiteLLM completion() model= gpt-5-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5-mini; provider = openai
14:23:43 - LiteLLM:INFO: utils.py:1273 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
14:23:44 - LiteLLM:INFO: utils.py:1273 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
14:23:45 - LiteLLM:INFO: utils.py:1273 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
2025/11/11 14:23:45 INFO mlf

Iteration 48: All subsample scores perfect. Skipping.
Iteration 48: Reflective mutation did not propose a new candidate
🏃 View run thundering-squid-391 at: http://127.0.0.1:5001/#/experiments/647522264572251786/runs/841edca585cc48fa973dab58f8ee54da
🧪 View experiment at: http://127.0.0.1:5001/#/experiments/647522264572251786
Optimized template: You are given a single sentence from a medical research paper. Classify it into exactly one of these section labels:
- CONCLUSIONS
- RESULTS
- METHODS
- OBJECTIVE
- BACKGROUND

Input format
- You will receive one sentence as the value of {{sentence}}.
- The sentence may be a standalone statement from any section of a scientific/medical paper.

Output format
- Return only one of: CONCLUSIONS, RESULTS, METHODS, OBJECTIVE, BACKGROUND.
- Use ALL CAPS.
- Do not include explanations, punctuation, or extra whitespace. No leading or trailing spaces.

Core classification approach
- Rely on rhetorical function and lexical cues in the sentence.
- Apply the 

In [13]:
from rich import print
print(optimized_prompt.format(sentence="Some example sentence"))

You are given a single sentence from a medical research paper. Classify it into exactly one of these section 
labels:
- CONCLUSIONS
- RESULTS
- METHODS
- OBJECTIVE
- BACKGROUND

Input format
- You will receive one sentence as the value of Some example sentence.
- The sentence may be a standalone statement from any section of a scientific/medical paper.

Output format
- Return only one of: CONCLUSIONS, RESULTS, METHODS, OBJECTIVE, BACKGROUND.
- Use ALL CAPS.
- Do not include explanations, punctuation, or extra whitespace. No leading or trailing spaces.

Core classification approach
- Rely on rhetorical function and lexical cues in the sentence.
- Apply the decision rules below in order, top to bottom. Assign the first matching label and stop.
- If none match, default to BACKGROUND.
- Favor earlier rules over later ones when a sentence could fit multiple categories.

Decision rules and cues (apply in order)

1) OBJECTIVE
   - Purpose/aim statements describing what the study intends to do or evaluate.
   - Strong cues:
     - “Objective:”, “Objectives:”, “Aim:”, “Aims:”, “Purpose:”
     - “The aim/objective of this study was…”
     - “We aimed/sought/intended to…”
     - “This study aimed to…”
     - Infinitive-purpose statements, especially at the start: “To assess/evaluate/determine/compare/investigate…”
     - Hypotheses framed as aims: “We hypothesized that…”
   - Note: If a sentence begins with “To …” and describes an evaluation or comparison the study intends to perform,
classify as OBJECTIVE.

2) METHODS
   - Study design, participants, procedures, interventions, materials, exposures, outcome measures, timelines, and 
analysis.
   - Strong cues:
     - Design: randomized, cohort, cross-sectional, case-control, double-blind, placebo-controlled, pragmatic 
trial.
     - Recruitment/eligibility, settings/sites, sample size (n=), allocation/randomization/blinding, trial 
registration, ethics/approvals/consent.
     - Intervention/program descriptions: theoretical basis, components, delivery mode, frequency/duration, dosage,
content.
     - Outcomes/measures/instruments: “Primary/secondary outcomes include…”, “Measures included…”, named scales 
(e.g., PHQ-9), imaging/lab methods.
     - Timing: “at baseline and 6 months,” “for 12 weeks,” session frequency.
     - Procedures: “Participants will…/were instructed to…”, data collection steps.
     - Analysis: statistical tests/models, adjustment/covariates, software, handling of missing data, power/sample 
size calculation.
   - Notes:
     - Future tense describing planned procedures typically indicates METHODS.
     - Meta-descriptions like “This paper describes the design and evaluation of X” are NOT METHODS (see 
BACKGROUND), unless the sentence explicitly states the study’s aim (then OBJECTIVE).

3) RESULTS
   - Empirical findings, observed data, and quantitative/qualitative outcomes.
   - Strong cues:
     - Numerical results, changes over time, differences between groups.
     - Effect sizes, percentages, means/SD, medians/IQR, CIs, p-values, hazard/odds/risk ratios, beta coefficients.
     - Verbs like “increased/decreased/improved/worsened,” “was associated with,” “we found/observed/identified.”
     - Reporting baseline characteristics with numerical values (e.g., “At baseline, mean age was 54.3 years”) → 
RESULTS.
   - If a sentence mixes findings and interpretation, prefer RESULTS if it contains specific data/findings.

4) CONCLUSIONS
   - Interpretive takeaways, implications, recommendations, or overarching statements of 
effectiveness/safety/utility.
   - Strong cues:
     - “Conclusions:”, “In conclusion,” “In summary,” “Overall,” “Taken together,” “These findings 
suggest/indicate/support…”
     - Practice/policy/clinical implications, recommendations, or statements of effectiveness/safety.
     - Statements about contribution/impact: “The results will inform…”, “These data provide evidence for…”, 
“Findings may guide…”
     - Future-oriented implications without pr

In [15]:
prompt.format(sentence="Some example sentence")

'Classify this medical research paper sentence into one of these sections: CONCLUSIONS, RESULTS, METHODS, OBJECTIVE, BACKGROUND.\n\nSentence: Some example sentence'